<a href="https://colab.research.google.com/github/ArmFriiz/Dicoding-Submission-FDL/blob/main/Analisis%20Sentimen/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Muhammad Faris Akbar**
<br></br>
**Fundamental Deep Learning - Sentimen Analisis Shopee**

In [1]:
!pip install google-play-scraper transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.4 MB/s eta 0:00:00


In [2]:
import sys
import os
import pandas as pd
import re

In [3]:
# try:
#     # Jika dijalankan sebagai file .py
#     base_path = os.path.dirname(os.path.abspath(__file__))
# except NameError:
#     # Jika dijalankan di Jupyter/Interactive
#     base_path = os.getcwd()

# parent_dir = os.path.abspath(os.path.join(base_path, '..'))
# sys.path.append(parent_dir)

# print("Base Path:", base_path)
# print("Parent Directory:", parent_dir)

# from Helper import *

In [4]:
from google_play_scraper import reviews, Sort
from transformers import pipeline
from tqdm import tqdm

**Helper Function**

In [5]:
def cek_nan(data):
  """
  Function yang digunakan untuk memeriksa nilai nan dari data
  input   : data
  output  : nan_info (DataFrame)
  return  : nan_info (DataFrame)
  """
  nan_info = pd.DataFrame(data.isna().sum().sort_values(ascending=False), columns=['Jumlah Nilai Missing'])

  if nan_info['Jumlah Nilai Missing'].sum() == 0:
    print("Tidak ada nilai missing")
    return None
  else:
    nan_info['Persentase Nilai Missing (%)'] = ((nan_info['Jumlah Nilai Missing'] / len(data)) * 100).round(3)

    # return data dengan nilai missing, apabila ingin mereturn keseluruhan data maka ubah menjadi return nan_info
    return nan_info[nan_info['Jumlah Nilai Missing'] > 0]

In [6]:
def visualize_row_with_nan(data, method='all', columns=None):
  """
  Procedure yang digunakan untuk menvisualisasikan baris yang mengandung NaN berdasarkan metode yang dipilih
  input   : data, method, columns
  output  : baris_nan (DataFrame)
  """
  try:
    if method not in ['all', 'column', 'columns']:
      raise ValueError("Metode tidak valid. Pilih salah satu dari 'all', 'column', 'columns'.")

    if method == 'all':
      display(data[data.isna().any(axis=1)])

    elif method == 'column':
      if not isinstance(columns, str):
          raise TypeError("Untuk metode 'column', argumen 'columns' harus berupa String.")
      if columns not in data.columns:
          raise KeyError(f"Kolom '{columns}' tidak ditemukan dalam DataFrame.")
      display(data[data[columns].isna()])

    elif method == 'columns':
      if not isinstance(columns, list):
          raise TypeError("Untuk metode 'columns', argumen 'columns' harus berupa list.")
      for column in columns:
          if column not in data.columns:
              raise KeyError(f"Kolom '{column}' tidak ditemukan dalam DataFrame.")
      display(data[data[columns].isna().all(axis=1)])

  except (TypeError, KeyError) as e:
    print(f"Kesalahan dalam memproses data: {e}")

In [7]:
def visualize_row_with_duplicated(data):
  """
  Procedure yang digunakan untuk menvisualisasikan baris yang mengandung data duplikat
  input   : data
  output  : baris_duplikat (DataFrame)
  """
  duplicated = data.duplicated().sum()

  if duplicated > 0:
    print("Jumlah Data Duplikat :", duplicated)
    all_duplicates = data[data.duplicated(keep=False)]

    duplicate_indices = all_duplicates.groupby(list(all_duplicates.columns)).groups
    print("Pasangan Data Duplikat :")
    for group_indices in duplicate_indices.values():
      if len(group_indices) > 1:
        display(data.iloc[list(group_indices)])
        print("\n")
  else:
    print("Tidak ada data duplikat")

**Main Section**

In [8]:
def scrape_google_play(app_id, total_count, country='id', lang='id'):
    """
    Fungsi untuk melakukan scraping ulasan dari Google Play Store.

    app_id (str): ID aplikasi di Play Store (contoh: 'com.shopee.id')
    total_count (int): Target jumlah data yang ingin diambil
    country (str): Kode negara (default Indonesia 'id')
    lang (str): Bahasa ulasan (default Indonesia 'id')
    """
    print(f"Proses scraping untuk aplikasi: {app_id}...")

    result, continuation_token = reviews(
        app_id,
        lang=lang,
        country=country,
        sort=Sort.NEWEST, # Sort.NEWEST digunakan untuk mendapat data terbaru
        count=total_count,
        filter_score_with=None # Mengambil semua rating (1-5)
    )

    print(f"Berhasil mengambil {len(result)} data mentah.")

    # Konversi hasil scraping ke dalam DataFrame (Tabel)
    df = pd.DataFrame(result)

    return df

In [9]:
TARGET_APP = 'com.shopee.id'
JUMLAH_DATA = 5000

df_ulasan = scrape_google_play(TARGET_APP, JUMLAH_DATA)

Proses scraping untuk aplikasi: com.shopee.id...
Berhasil mengambil 5000 data mentah.


In [10]:
# df_ulasan = pd.read_csv('https://raw.githubusercontent.com/ArmFriiz/Dicoding-Submission-FDL/refs/heads/main/Analisis%20Sentimen/dataset_ulasan_playstore.csv')

In [11]:
df_ulasan.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,1c630f10-f578-4425-8836-1098530bb01f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,siiip..,5,0,3.67.25,2026-01-28 07:44:28,"Hi kak Heryanto Yusuf, makasih buat review bin...",2026-01-28 08:08:51,3.67.25
1,88526ce0-4a8c-4fe4-9c09-bb3a6785a12e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dengan adanya jasa pengiriman di bawah naungan...,1,0,3.66.27,2026-01-28 07:43:29,"Hai kak Qinenk Tj, mohon maaf atas ketidaknyam...",2026-01-28 08:15:32,3.66.27
2,6c159037-ed16-4166-a0b6-f31025de419b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,5,0,None,2026-01-28 07:43:28,"Hi Kak Yudi Hartono, makasih ya buat bintang 5...",2026-01-28 08:07:43,None
3,dfa14d20-03a3-45fa-907e-29a839b58af7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,mantap,5,0,3.67.25,2026-01-28 07:42:49,Hai kak it's me Rapzz. Maaf terkait kendala pe...,2025-05-23 06:12:11,3.67.25
4,f9cdba56-cd79-4cee-a603-7954a4e3c537,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"over all uda bgs, tp tlg tambahin gopay dong m...",4,0,None,2026-01-28 07:41:59,Wuiihhh makasih banyak review dan bintangnya :...,2026-01-28 08:09:56,None


In [12]:
df_ulasan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              5000 non-null   object        
 1   userName              5000 non-null   object        
 2   userImage             5000 non-null   object        
 3   content               5000 non-null   object        
 4   score                 5000 non-null   int64         
 5   thumbsUpCount         5000 non-null   int64         
 6   reviewCreatedVersion  3868 non-null   object        
 7   at                    5000 non-null   datetime64[ns]
 8   replyContent          4832 non-null   object        
 9   repliedAt             4832 non-null   datetime64[ns]
 10  appVersion            3868 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 429.8+ KB


In [13]:
df_ulasan.describe(include='all')

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
count,5000,5000,5000,5000,5000.000000,5000.000000,3868,5000,4832,4832,3868
unique,5000,496,498,4032,NaN,NaN,139,NaN,4706,NaN,139
top,b95255e2-3381-4088-a02a-f6767979e794,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,NaN,NaN,3.66.27,NaN,"Hai Kak , makasih ya buat penilaiannya, semoga...",NaN,3.66.27
freq,1,4504,4503,165,NaN,NaN,2271,NaN,72,NaN,2271
mean,NaN,NaN,NaN,NaN,4.115200,2.559000,NaN,2026-01-25 07:37:25.935800064,NaN,2025-12-21 18:54:08.749171968,NaN
min,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,2026-01-22 11:01:25,NaN,2020-11-18 09:06:01,NaN
25%,NaN,NaN,NaN,NaN,4.000000,0.000000,NaN,2026-01-23 22:58:31.500000,NaN,2026-01-23 14:11:27.750000128,NaN
50%,NaN,NaN,NaN,NaN,5.000000,0.000000,NaN,2026-01-25 06:51:57.500000,NaN,2026-01-25 04:14:12.500000,NaN
75%,NaN,NaN,NaN,NaN,5.000000,0.000000,NaN,2026-01-26 14:56:57.500000,NaN,2026-01-26 13:42:58.249999872,NaN
max,NaN,NaN,NaN,NaN,5.000000,8284.000000,NaN,2026-01-28 07:44:28,NaN,2026-01-28 12:57:14,NaN


**Cek Validitas dan Kebersihan Data**

In [14]:
cek_nan(df_ulasan)

,Jumlah Nilai Missing,Persentase Nilai Missing (%)
reviewCreatedVersion,1132,22.64
appVersion,1132,22.64
replyContent,168,3.36
repliedAt,168,3.36


In [15]:
visualize_row_with_nan(df_ulasan)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
2,6c159037-ed16-4166-a0b6-f31025de419b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,5,0,None,2026-01-28 07:43:28,"Hi Kak Yudi Hartono, makasih ya buat bintang 5...",2026-01-28 08:07:43,None
4,f9cdba56-cd79-4cee-a603-7954a4e3c537,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"over all uda bgs, tp tlg tambahin gopay dong m...",4,0,None,2026-01-28 07:41:59,Wuiihhh makasih banyak review dan bintangnya :...,2026-01-28 08:09:56,None
14,45b1a79b-aa5f-4e1d-bcd1-8b357257003f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,uang jualan selalu di tilep shopee,1,0,None,2026-01-28 07:35:46,"Hi kak Nugroho Nusantoro, maaf banget ya udah ...",2026-01-28 08:10:20,None
15,cb767cb6-76ef-41ed-aae9-0e6ac5a2443a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,barang tiba tepat waktu dan ordersnnya sesuai,5,0,None,2026-01-28 07:35:33,Hallo kak Suhatim Suhatim. Makasih yaa buat re...,2026-01-28 08:07:43,None
18,c8dc9394-0a8b-4006-af40-08020600cc07,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat mantap.mempermudah urusan financial saya,5,0,None,2026-01-28 07:33:03,"Hi kak Syamsoel Boy, makasih ya buat bintang 5...",2026-01-28 08:08:41,None
...,...,...,...,...,...,...,...,...,...,...,...
4980,ce0fb20c-e105-4d17-b0c1-49dd8e97784a,Indri Jon istovel,https://play-lh.googleusercontent.com/a-/ALV-U...,Shopee mantab,5,0,None,2026-01-22 11:17:35,"Hai Kak , makasih ya buat penilaiannya, semoga...",2026-01-22 13:35:58,None
4983,d28174e4-258c-49a8-91c0-bc24d54aa64d,Helmi Alfariz,https://play-lh.googleusercontent.com/a/ACg8oc...,sangat memuaskan dalam berbelanja segala kebut...,5,0,None,2026-01-22 11:15:47,Hallo kak Helmi Alfariz. Makasih yaa buat revi...,2026-01-22 13:40:40,None
4985,68ab749f-7f5d-40fc-ba36-cd61c572bbf3,Indriyani 17,https://play-lh.googleusercontent.com/a/ACg8oc...,iklanny ada di mna' gedeg bngt tiba' masuk k a...,3,0,None,2026-01-22 11:15:29,"Hi kak Indriyani 17, mohon maaf ya kak atas ke...",2026-01-22 13:36:48,None
4991,4ca309b3-da0b-4ae0-82d9-fc0b9639d49f,Wisonggeni,https://play-lh.googleusercontent.com/a-/ALV-U...,👍,5,0,None,2026-01-22 11:09:29,"Hai Kak , makasih ya buat penilaiannya, semoga...",2026-01-22 13:38:56,None


In [16]:
visualize_row_with_duplicated(df_ulasan)

Tidak ada data duplikat


**Filter kolom content dan score untuk konten data**

In [17]:
df = df_ulasan[['content', 'score']]

In [18]:
visualize_row_with_duplicated(df)

Jumlah Data Duplikat : 921
Pasangan Data Duplikat :


,content,score
424,Alhamdulillah,5
826,Alhamdulillah,5
1829,Alhamdulillah,5
3009,Alhamdulillah,5


,content,score
232,Bagus,5
663,Bagus,5
719,Bagus,5
1442,Bagus,5
1739,Bagus,5
1808,Bagus,5
1925,Bagus,5
3203,Bagus,5
3694,Bagus,5
3761,Bagus,5


,content,score
354,Baik,5
1563,Baik,5
3502,Baik,5
4318,Baik,5


,content,score
1519,Good,5
1683,Good,5
3198,Good,5
3636,Good,5
4647,Good,5


...

,content,score
3072,top markotop,5
3703,top markotop,5
4795,top markotop,5


,content,score
1835,very good,5
2252,very good,5


,content,score
1160,👍,5
1372,👍,5
2381,👍,5
3528,👍,5
3776,👍,5
4154,👍,5
4290,👍,5
4457,👍,5
4991,👍,5


,content,score
3105,👍🏻,5
3113,👍🏻,5


,content,score
1216,👍👍👍,5
1263,👍👍👍,5
2414,👍👍👍,5
3085,👍👍👍,5
3456,👍👍👍,5


In [19]:
df.drop_duplicates(inplace=True)

/tmp/ipython-input-3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [20]:
visualize_row_with_duplicated(df)

Tidak ada data duplikat


In [21]:
print(f"Ukuran Data Setelah Pembersihan NaN dan Duplicated: {df.shape}")

Ukuran Data Setelah Pembersihan NaN dan Duplicated: (4079, 2)


**Soft Cleaning Data**

In [22]:
def cleaning_untuk_labeling(text):
    text = str(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Hapus URL
    text = re.sub(r'<.*?>', '', text) # Hapus HTML tags
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Hapus Mentions (@user)
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
    text = re.sub(r'(.)\1{2,}', r'\1\1', text) # Hapus kata berulang yang muncul lebih dari 2x
    text = re.sub(r'[a-zA-Z]+\d+\w*|\w*\d+[a-zA-Z]+', '', text) # Hapus kombinasi angka dan huruf seperti m4ndi, 4yam, dll
    text = re.sub(r'\b\d{7,}\b', '', text) # Hapus angka yang panjangnya lebih dari 7
    text = ' '.join(text.split())

    return text

In [23]:
df['soft_clean_content'] = df['content'].apply(cleaning_untuk_labeling)

/tmp/ipython-input-3141896218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['soft_clean_content'] = df['content'].apply(cleaning_untuk_labeling)


In [24]:
df.head(5)

,content,score,soft_clean_content
0,siiip..,5,siip..
1,Dengan adanya jasa pengiriman di bawah naungan...,1,Dengan adanya jasa pengiriman di bawah naungan...
2,bagus,5,bagus
3,mantap,5,mantap
4,"over all uda bgs, tp tlg tambahin gopay dong m...",4,"over all uda bgs, tp tlg tambahin gopay dong m..."


In [25]:
cek_nan(df)

Tidak ada nilai missing


In [26]:
visualize_row_with_nan(df)

,content,score,soft_clean_content


In [27]:
df.dropna(inplace=True)

/tmp/ipython-input-1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


**Labeling Data**

In [28]:
# def labeling_data(df):
#     """
#     Fungsi untuk memberikan label otomatis berdasarkan skor bintang.
#     Menggunakan logika:
#     1-2 Bintang = Negatif
#     3 Bintang   = Netral
#     4-5 Bintang = Positif
#     """
#     def get_sentiment(score):
#         if score <= 2:
#             return 'Negatif'
#         elif score == 3:
#             return 'Netral'
#         else:
#             return 'Positif'

#     # Terapkan fungsi get_sentiment ke kolom 'score'
#     df['label'] = df['score'].apply(get_sentiment)

#     return df

In [29]:
# print("Labeling data berdasarkan skor bintang")
# df_labeled = labeling_data(df)

In [30]:
def label_with_indobert(df):
  pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

  nlp = pipeline(
      "sentiment-analysis",
      model=pretrained_name,
      tokenizer=pretrained_name,
      truncation=True, # Potong teks jika terlalu panjang (>512 kata)
      max_length=512
    )

  labels = []
  scores = []

  for text in tqdm(df['soft_clean_content']):
    try:
      result = nlp(text)[0] # Prediksi sentimen

      label = result['label'] # output: 'positive', 'neutral', 'negative'

      # Mapping ulang ke format Bahasa Indonesia
      label_map = {
        'positive': 'Positif',
        'neutral': 'Netral',
        'negative': 'Negatif'
      }
      labels.append(label_map.get(label, label))
      scores.append(result['score'])

    except Exception as e:
      print(f"Error pada teks: {text}")
      labels.append("Netral")
      scores.append(0.0)

  df['sentiment_label'] = labels
  df['confidence_score'] = scores

  return df

In [ ]:
df_labeled = label_with_indobert(df)

In [32]:
df_labeled.head(5)

,content,score,soft_clean_content,sentiment_label,confidence_score
0,siiip..,5,siip..,Negatif,0.998343
1,Dengan adanya jasa pengiriman di bawah naungan...,1,Dengan adanya jasa pengiriman di bawah naungan...,Negatif,0.569122
2,bagus,5,bagus,Positif,0.998097
3,mantap,5,mantap,Positif,0.999183
4,"over all uda bgs, tp tlg tambahin gopay dong m...",4,"over all uda bgs, tp tlg tambahin gopay dong m...",Netral,0.789292


**Pemeriksaan Distribusi Data, memastikan apakah terdapat imbalance atau tidak**

In [33]:
print("Distribusi Data per Kelas:")
print(df_labeled['sentiment_label'].value_counts())

Distribusi Data per Kelas:
sentiment_label
Positif    2478
Negatif    1376
Netral      225
Name: count, dtype: int64


**Konversi ke csv untuk mempermudah dalam pembersihan data lebih lanjut**

In [34]:
nama_file = 'dataset_ulasan_playstore.csv'
df_labeled.to_csv(nama_file, index=False)

print(f"Selesai! Data berhasil disimpan ke '{nama_file}'")
print(f"Total data: {len(df_labeled)}")

Selesai! Data berhasil disimpan ke 'dataset_ulasan_playstore.csv'
Total data: 4079
